In [1]:
# 📦 1. Imports
from bs4 import BeautifulSoup
from lxml import etree
from pathlib import Path
from langchain_core.documents import Document
from typing import List, Dict

In [2]:
# 📁 2. Chargement des fichiers
BASE_PATH = "../../data_collection/data/bofip/BOFiP/documents/Contenu/Commentaire/AIS/12628-PGP/2024-07-10"
html_path = Path(BASE_PATH + "/" + "data.html")
xml_path = Path(BASE_PATH + "/" +  "document.xml")
html_raw = html_path.read_text(encoding="utf-8")

In [3]:
# 🧠 3. Fonctions d'extraction

def extract_document_title(html: str) -> str:
    soup = BeautifulSoup(html, "html.parser")
    title_tag = soup.find("title")
    return title_tag.get_text(strip=True) if title_tag else ""

def extract_metadata(xml_path: Path) -> Dict:
    tree = etree.parse(str(xml_path))
    ns = {'dc': 'http://purl.org/dc/elements/1.1'}
    relations = tree.findall('.//dc:relation', namespaces=ns)
    referenced_ids = [rel.text for rel in relations if rel.get("type") == "references"]

    return {
        "title_xml": tree.findtext('.//dc:title', namespaces=ns),
        "date": tree.findtext('.//dc:date', namespaces=ns),
        "source": tree.findtext('.//dc:source', namespaces=ns),
        "url": tree.findall('.//dc:identifier', namespaces=ns)[-1].text,
        "file": xml_path.stem,
        "references": referenced_ids
    }

In [4]:
# 🧱 4. Extraction des sections depuis le HTML

def extract_sections(html: str) -> List[Dict]:
    soup = BeautifulSoup(html, "html.parser")
    tags = soup.find_all(['h1', 'h2', 'h3', 'p', 'ul', 'ol'])

    sections = []
    current = {
        "title": "Préambule",
        "level": 0,
        "content": ""
    }

    for tag in tags:
        if tag.name in ["h1", "h2", "h3"]:
            if current["content"].strip():
                sections.append(current)
            current = {
                "title": tag.get_text(strip=True),
                "level": int(tag.name[1]),
                "content": ""
            }
        else:
            current["content"] += "\n" + tag.get_text(separator=" ", strip=True)

    if current["content"].strip():
        sections.append(current)

    return sections

In [5]:
# 📄 5. Création des Documents LangChain

def html_to_documents(html_text: str, xml_path: Path) -> List[Document]:
    metadata_common = extract_metadata(xml_path)
    metadata_common["title_head"] = extract_document_title(html_text)

    sections = extract_sections(html_text)

    return [
        Document(
            page_content=section["content"],
            metadata={
                **metadata_common,
                "section": section["title"],
                "level": section["level"]
            }
        )
        for section in sections if section["content"].strip()
    ]

In [6]:
# 🚀 6. Exécution du pipeline

docs = html_to_documents(html_raw, xml_path)

print(f"{len(docs)} documents générés")
print(docs[0].metadata)
print(docs[0].page_content[:500])

1 documents générés
{'title_xml': 'AIS - Mobilités - Taxes sur les déplacements routiers', 'date': '2024-07-10', 'source': 'Bulletin Officiel des Finances Publiques - Impôts', 'url': 'https://bofip.impots.gouv.fr/bofip/12628-PGP.html/identifiant=BOI-AIS-MOB-10-20240710', 'file': 'document', 'references': ['Actualite:13887-PGP', 'Contenu:13890-PGP', 'Contenu:13911-PGP', 'Contenu:13931-PGP', 'Contenu:14231-PGP'], 'title_head': 'AIS - Mobilités - Taxes sur les déplacements routiers', 'section': 'Préambule', 'level': 0}

Actualité liée : [node:date:13887-PGP] : AIS - TFP - ENR - Consultation publique - Recodification des taxes sur les déplacements routiers au sein du code des impositions sur les biens et services
1
Le présent titre commente les différentes taxes portant sur les déplacements routiers, régies par le chapitre Ier du titre II du livre IV du code des impositions sur les biens et services depuis le 1 er janvier 2022.
10
Sont traitées successivement :
les dispositions générales (

In [7]:
# 💾 7. Export en JSONL pour vectorisation
import json

with open("bofip_documents.jsonl", "w", encoding="utf-8") as f:
    for doc in docs:
        json.dump({
            "page_content": doc.page_content,
            "metadata": doc.metadata
        }, f, ensure_ascii=False)
        f.write("\n")
